In [ ]:
from sympy import * 
ux, uy, sx, sy, sxy, c1, c2, alpha = symbols('u_x u_y s_x s_y s_{xy} c1 c2 alpha')

In [ ]:
ssim = (2*ux*uy*alpha + c1) * (2*sxy*alpha + c2) / (ux**2 + alpha**2 * uy**2 + c1) / (sx**2 + alpha**2 * sy**2 + c2)
ssim

In [ ]:
derivative = diff(ssim, alpha)
derivative

In [ ]:
from sympy import * 
# ux, uy, sx, sy, sxy, alpha = symbols('u_x u_y s_x s_y s_{xy} alpha', real=True, positive=True)
ux, uy, sx, sy, sxy = symbols('u_x u_y s_x s_y s_{xy}', real=True, positive=True)
alpha= symbols(' alpha', real=True)
ssim = (2*ux*uy*alpha ) * (2*sxy*alpha ) / (ux**2 + alpha**2 * uy**2 ) / (sx**2 + alpha**2 * sy**2 )
ssim



In [ ]:
derivative = diff(ssim, alpha)
derivative

In [ ]:
out = solve(derivative, alpha)
out

In [ ]:
derivative.subs(alpha, out[0]).simplify()

In [ ]:
double_derivative = diff(derivative, alpha)
double_derivative.subs(alpha, out[2]).simplify()

In [ ]:
double_derivative.subs(alpha, out[1]).simplify()

In [ ]:
latex(double_derivative.subs(alpha, out[0]).simplify())

In [ ]:
solve(derivative, 0, alpha)

## Solving it with numerical optimization

In [ ]:
from disentangle.core.tiff_reader import load_tiff
pred_fpath = '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8_F/kth_0/pred_training_disentangle_2406_D25-M3-S0-L8_12_1.tif'
pred = load_tiff(pred_fpath)

In [ ]:
import os
pred_fname =os.path.basename(pred_fpath)
assert pred_fname.endswith('_1.tif')
model_subdir = pred_fname.replace('pred_training_disentangle_', '').replace('_1.tif', '').replace('_','/')
model_fpath = os.path.join('/group/jug/ashesh/training/disentangle/', model_subdir)
model_fpath 

In [ ]:
from disentangle.config_utils import load_config
from disentangle.core.data_split_type import DataSplitType
from disentangle.data_loader.nikola_7D_rawdata_loader import get_train_val_data
from ml_collections import ConfigDict
config = ConfigDict(load_config(model_fpath))
with config.unlocked():
    config.data.dset_type = '500ms'
datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
gt_data = get_train_val_data(datadir, config.data, DataSplitType.Test,
                            config.training.val_fraction, config.training.test_fraction)


In [ ]:
kth_idx = int(pred_fpath.split('/')[-2].split('_')[-1])
tar = gt_data[kth_idx:kth_idx+1]
tar = tar[:,:pred.shape[1], :pred.shape[2], :pred.shape[3]]

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(tar[0,...,0])
ax[1].imshow(pred[0,...,0])

In [ ]:
from disentangle.core.ssim import compute_multiscale_ssim
import numpy as np
compute_multiscale_ssim(tar.astype(np.float32), pred.astype(np.float32))

In [ ]:
compute_multiscale_ssim(tar.astype(np.float32), pred.astype(np.float32), range_invariant=False)

In [ ]:
import functools

import numpy as np
from scipy.ndimage import uniform_filter

from skimage._shared import utils
from skimage._shared.filters import gaussian
from skimage._shared.utils import _supported_float_type, check_shape_equality, warn
from skimage.util.arraycrop import crop
from skimage.util.dtype import dtype_range

__all__ = ['structural_similarity']


def structural_similarity_dict(
    im1,
    im2,
    *,
    win_size=None,
    gradient=False,
    data_range=None,
    channel_axis=None,
    gaussian_weights=False,
    full=False,
    **kwargs,
):
    """
    Compute the mean structural similarity index between two images.
    Please pay attention to the `data_range` parameter with floating-point images.

    Parameters
    ----------
    im1, im2 : ndarray
        Images. Any dimensionality with same shape.
    win_size : int or None, optional
        The side-length of the sliding window used in comparison. Must be an
        odd value. If `gaussian_weights` is True, this is ignored and the
        window size will depend on `sigma`.
    gradient : bool, optional
        If True, also return the gradient with respect to im2.
    data_range : float, optional
        The data range of the input image (difference between maximum and
        minimum possible values). By default, this is estimated from the image
        data type. This estimate may be wrong for floating-point image data.
        Therefore it is recommended to always pass this scalar value explicitly
        (see note below).
    channel_axis : int or None, optional
        If None, the image is assumed to be a grayscale (single channel) image.
        Otherwise, this parameter indicates which axis of the array corresponds
        to channels.

        .. versionadded:: 0.19
           ``channel_axis`` was added in 0.19.
    gaussian_weights : bool, optional
        If True, each patch has its mean and variance spatially weighted by a
        normalized Gaussian kernel of width sigma=1.5.
    full : bool, optional
        If True, also return the full structural similarity image.

    Other Parameters
    ----------------
    use_sample_covariance : bool
        If True, normalize covariances by N-1 rather than, N where N is the
        number of pixels within the sliding window.
    K1 : float
        Algorithm parameter, K1 (small constant, see [1]_).
    K2 : float
        Algorithm parameter, K2 (small constant, see [1]_).
    sigma : float
        Standard deviation for the Gaussian when `gaussian_weights` is True.

    Returns
    -------
    mssim : float
        The mean structural similarity index over the image.
    grad : ndarray
        The gradient of the structural similarity between im1 and im2 [2]_.
        This is only returned if `gradient` is set to True.
    S : ndarray
        The full SSIM image.  This is only returned if `full` is set to True.

    Notes
    -----
    If `data_range` is not specified, the range is automatically guessed
    based on the image data type. However for floating-point image data, this
    estimate yields a result double the value of the desired range, as the
    `dtype_range` in `skimage.util.dtype.py` has defined intervals from -1 to
    +1. This yields an estimate of 2, instead of 1, which is most often
    required when working with image data (as negative light intensities are
    nonsensical). In case of working with YCbCr-like color data, note that
    these ranges are different per channel (Cb and Cr have double the range
    of Y), so one cannot calculate a channel-averaged SSIM with a single call
    to this function, as identical ranges are assumed for each channel.

    To match the implementation of Wang et al. [1]_, set `gaussian_weights`
    to True, `sigma` to 1.5, `use_sample_covariance` to False, and
    specify the `data_range` argument.

    .. versionchanged:: 0.16
        This function was renamed from ``skimage.measure.compare_ssim`` to
        ``skimage.metrics.structural_similarity``.

    References
    ----------
    .. [1] Wang, Z., Bovik, A. C., Sheikh, H. R., & Simoncelli, E. P.
       (2004). Image quality assessment: From error visibility to
       structural similarity. IEEE Transactions on Image Processing,
       13, 600-612.
       https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf,
       :DOI:`10.1109/TIP.2003.819861`

    .. [2] Avanaki, A. N. (2009). Exact global histogram specification
       optimized for structural similarity. Optical Review, 16, 613-621.
       :arxiv:`0901.0065`
       :DOI:`10.1007/s10043-009-0119-z`

    """
    check_shape_equality(im1, im2)
    float_type = _supported_float_type(im1.dtype)

    if channel_axis is not None:
        # loop over channels
        args = dict(
            win_size=win_size,
            gradient=gradient,
            data_range=data_range,
            channel_axis=None,
            gaussian_weights=gaussian_weights,
            full=full,
        )
        args.update(kwargs)
        nch = im1.shape[channel_axis]
        mssim = np.empty(nch, dtype=float_type)

        if gradient:
            G = np.empty(im1.shape, dtype=float_type)
        if full:
            S = np.empty(im1.shape, dtype=float_type)
        channel_axis = channel_axis % im1.ndim
        _at = functools.partial(utils.slice_at_axis, axis=channel_axis)
        for ch in range(nch):
            ch_result = structural_similarity(im1[_at(ch)], im2[_at(ch)], **args)
            if gradient and full:
                mssim[ch], G[_at(ch)], S[_at(ch)] = ch_result
            elif gradient:
                mssim[ch], G[_at(ch)] = ch_result
            elif full:
                mssim[ch], S[_at(ch)] = ch_result
            else:
                mssim[ch] = ch_result
        mssim = mssim.mean()
        if gradient and full:
            return mssim, G, S
        elif gradient:
            return mssim, G
        elif full:
            return mssim, S
        else:
            return mssim

    K1 = kwargs.pop('K1', 0.01)
    K2 = kwargs.pop('K2', 0.03)
    sigma = kwargs.pop('sigma', 1.5)
    if K1 < 0:
        raise ValueError("K1 must be positive")
    if K2 < 0:
        raise ValueError("K2 must be positive")
    if sigma < 0:
        raise ValueError("sigma must be positive")
    use_sample_covariance = kwargs.pop('use_sample_covariance', True)

    if gaussian_weights:
        # Set to give an 11-tap filter with the default sigma of 1.5 to match
        # Wang et. al. 2004.
        truncate = 3.5

    if win_size is None:
        if gaussian_weights:
            # set win_size used by crop to match the filter size
            r = int(truncate * sigma + 0.5)  # radius as in ndimage
            win_size = 2 * r + 1
        else:
            win_size = 7  # backwards compatibility

    if np.any((np.asarray(im1.shape) - win_size) < 0):
        raise ValueError(
            'win_size exceeds image extent. '
            'Either ensure that your images are '
            'at least 7x7; or pass win_size explicitly '
            'in the function call, with an odd value '
            'less than or equal to the smaller side of your '
            'images. If your images are multichannel '
            '(with color channels), set channel_axis to '
            'the axis number corresponding to the channels.'
        )

    if not (win_size % 2 == 1):
        raise ValueError('Window size must be odd.')

    if data_range is None:
        if np.issubdtype(im1.dtype, np.floating) or np.issubdtype(
            im2.dtype, np.floating
        ):
            raise ValueError(
                'Since image dtype is floating point, you must specify '
                'the data_range parameter. Please read the documentation '
                'carefully (including the note). It is recommended that '
                'you always specify the data_range anyway.'
            )
        if im1.dtype != im2.dtype:
            warn(
                "Inputs have mismatched dtypes. Setting data_range based on im1.dtype.",
                stacklevel=2,
            )
        dmin, dmax = dtype_range[im1.dtype.type]
        data_range = dmax - dmin
        if np.issubdtype(im1.dtype, np.integer) and (im1.dtype != np.uint8):
            warn(
                "Setting data_range based on im1.dtype. "
                + f"data_range = {data_range:.0f}. "
                + "Please specify data_range explicitly to avoid mistakes.",
                stacklevel=2,
            )

    ndim = im1.ndim

    if gaussian_weights:
        filter_func = gaussian
        filter_args = {'sigma': sigma, 'truncate': truncate, 'mode': 'reflect'}
    else:
        filter_func = uniform_filter
        filter_args = {'size': win_size}

    # ndimage filters need floating point data
    im1 = im1.astype(float_type, copy=False)
    im2 = im2.astype(float_type, copy=False)

    NP = win_size**ndim

    # filter has already normalized by NP
    if use_sample_covariance:
        cov_norm = NP / (NP - 1)  # sample covariance
    else:
        cov_norm = 1.0  # population covariance to match Wang et. al. 2004

    # compute (weighted) means
    ux = filter_func(im1, **filter_args)
    uy = filter_func(im2, **filter_args)

    # compute (weighted) variances and covariances
    uxx = filter_func(im1 * im1, **filter_args)
    uyy = filter_func(im2 * im2, **filter_args)
    uxy = filter_func(im1 * im2, **filter_args)
    vx = cov_norm * (uxx - ux * ux)
    vy = cov_norm * (uyy - uy * uy)
    vxy = cov_norm * (uxy - ux * uy)

    R = data_range
    C1 = (K1 * R) ** 2
    C2 = (K2 * R) ** 2

    return {
        'ux': ux,
        'uy': uy,
        'vxy': vxy,
        'vx': vx,
        'vy': vy,
        'C1': C1,
        'C2': C2,
    }

    # A1, A2, B1, B2 = (
    #     2 * ux * uy + C1,
    #     2 * vxy + C2,
    #     ux**2 + uy**2 + C1,
    #     vx + vy + C2,
    # )
    
    # D = B1 * B2
    # S = (A1 * A2) / D

    # # to avoid edge effects will ignore filter radius strip around edges
    # pad = (win_size - 1) // 2

    # # compute (weighted) mean of ssim. Use float64 for accuracy.
    # mssim = crop(S, pad).mean(dtype=np.float64)

    # if gradient:
    #     # The following is Eqs. 7-8 of Avanaki 2009.
    #     grad = filter_func(A1 / D, **filter_args) * im1
    #     grad += filter_func(-S / B2, **filter_args) * im2
    #     grad += filter_func((ux * (A2 - A1) - uy * (B2 - B1) * S) / D, **filter_args)
    #     grad *= 2 / im1.size

    #     if full:
    #         return mssim, grad, S
    #     else:
    #         return mssim, grad
    # else:
    #     if full:
    #         return mssim, S
    #     else:
    #         return mssim

In [ ]:
def neg_ssim_from_params(alpha, ux, uy, vx, vy, vxy, C1, C2):
    A1, A2, B1, B2 = (
        2 * alpha* ux * uy + C1,
        2 * alpha * vxy + C2,
        ux**2 + (alpha**2) * uy**2 + C1,
        vx + (alpha**2)*vy + C2,
    )
    
    D = B1 * B2
    S = (A1 * A2) / D
    return np.mean(-1 * S)

In [ ]:
ssim_dict = structural_similarity_dict(tar[0,...,0].astype(np.float32), pred[0,...,0].astype(np.float32), data_range = tar[:,...,0].max() - tar[:,...,0].min())

In [ ]:
ssim_from_params(0.1, ssim_dict['ux'], ssim_dict['uy'],ssim_dict['vx'],ssim_dict['vy'], ssim_dict['vxy'], ssim_dict['C1'], ssim_dict['C2']).mean()

In [ ]:
vxy = ssim_dict['vxy']
vx = ssim_dict['vx']
vy = ssim_dict['vy']

ux = ssim_dict['ux']
uy = ssim_dict['uy']

C1 = ssim_dict['C1']
C2 = ssim_dict['C2']

In [ ]:
%%timeit 
from scipy.optimize import minimize
import numpy as np

# initial (bad) guess at (x,y) values
initial_guess = np.array([1])
res = minimize(neg_ssim_from_params, initial_guess, args=(ux, uy, vx, vy, vxy, C1, C2))

In [ ]:
res

In [ ]:
from disentangle.core.psnr import fix, zero_mean
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure
from disentangle.core.numpy_decorator import allow_numpy
import torch
from skimage.metrics import structural_similarity as ssim

def range_invariant_ssim(gt_, pred_):
    """
    Computes range invariant multiscale ssim for one channel.
    This has the benefit that it is invariant to scalar multiplications in the prediction.
    """

    shape = gt_.shape
    gt_ = torch.Tensor(gt_.reshape((shape[0],-1)))
    pred_ = torch.Tensor(pred_.reshape((shape[0],-1)))
    gt_ = zero_mean(gt_)
    pred_ = zero_mean(pred_)
    pred_ = fix(gt_, pred_)
    pred_ = pred_.reshape(shape).numpy()
    gt_ = gt_.reshape(shape).numpy()
    return ssim(gt_, pred_, data_range=gt_.max() - gt_.min())

In [ ]:
%%timeit
range_invariant_multiscale_ssim(tar[0,...,0].astype(np.float32), pred[0,...,0].astype(np.float32))

In [ ]:
# from sympy import * 

# alpha = symbols('alpha')

# vxy = Matrix(ssim_dict['vxy'])
# vx = Matrix(ssim_dict['vx'])
# vy = Matrix(ssim_dict['vy'])

# ux = Matrix(ssim_dict['ux'])
# uy = Matrix(ssim_dict['uy'])

# C1 = ssim_dict['C1']
# C2 = ssim_dict['C2']

In [ ]:
# C1 = ssim_dict['C1']
# C2 = ssim_dict['C2']

In [ ]:
# num =  (2*alpha*vxy + C2) * (2*alpha*ux * uy + C1)
# den = (alpha**2 * vy + vx + C2) * (ux**2 + uy**2 + C1)
# ssim = num/den